In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install captum
!pip install fvcore

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [50]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 3, 32, 32)
KERNEL_SIZES = [1, 1, 1]
PADDING = [0, 0, 0]
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=3
NUM_CLASSES=43
IN_CHANNELS=3
NUM_RUN=15

dataset_name = "Baseline GTSRB"
folder_path = '/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/' + dataset_name
filename = '/Baseline GTSRB 1*1_result.csv'
file_path = folder_path+filename
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/Baseline GTSRB/Baseline GTSRB 1*1_result.csv


# Model structure defination

In [51]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()
        stride = 1  # Common stride for all branches

        # Define branches with different kernel sizes and paddings
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[0], stride=stride, padding=PADDING[0]),
            nn.ReLU()
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[1], stride=stride, padding=PADDING[1]),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[2], stride=stride, padding=PADDING[2]),
            nn.ReLU()
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=stride, padding=1),
        )


        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(27648, num_classes)
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x


# Get dataset

In [52]:
from torchvision import datasets, transforms

# Define the transform with resizing to 64x64
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image to 64x64 pixels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the GTSRB dataset with the updated transform
train_dataset = datasets.GTSRB(root='.', split='train', download=True, transform=transform)
test_dataset = datasets.GTSRB(root='.', split='test', download=True, transform=transform)

# FLOP Count

In [53]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(IN_CHANNELS, NUM_CLASSES)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
flops_table = count_flops(inception_model, INPUT_SHAPE)
print(flops_table)

| module             | #parameters or shape   | #flops   |
|:-------------------|:-----------------------|:---------|
| model              | 1.189M                 | 1.263M   |
|  branch1.0         |  32                    |  24.576K |
|   branch1.0.weight |   (8, 3, 1, 1)         |          |
|   branch1.0.bias   |   (8,)                 |          |
|  branch2.0         |  32                    |  24.576K |
|   branch2.0.weight |   (8, 3, 1, 1)         |          |
|   branch2.0.bias   |   (8,)                 |          |
|  branch3.0         |  32                    |  24.576K |
|   branch3.0.weight |   (8, 3, 1, 1)         |          |
|   branch3.0.bias   |   (8,)                 |          |
|  fc.0              |  1.189M                |  1.189M  |
|   fc.0.weight      |   (43, 27648)          |          |
|   fc.0.bias        |   (43,)                |          |


# Train and attribution functions

train and eval function

In [54]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [55]:
def print_ig(test_loader, model, device, num_batches=5):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Dictionary to store cumulative attributions for each layer
    cumul_ig_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs = inputs.to(device)

        # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
        for layer_name, layer_module in model.named_modules():
            # Check if the layer is a Conv2d layer with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerIntegratedGradients for the layer
                lig = LayerIntegratedGradients(model, layer_module)

                # Compute the attributions for the current layer
                try:
                    attributions = lig.attribute(inputs, target=target_class.to(device))
                except Exception as e:
                    print(f"Error computing attributions for layer {layer_name}: {e}")
                    continue

                # Sum up the attributions for the current layer
                attr_sum = attributions.cpu().detach().numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_ig_attributions:
                    cumul_ig_attributions[layer_name] += attr_sum
                else:
                    cumul_ig_attributions[layer_name] = attr_sum

                # Free up memory
                del attributions, lig

    # Calculate and print the average attributions for each layer
    avg_ig_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_ig_attributions.items()}
    for layer_name, avg_attr in avg_ig_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_ig_attributions


def print_deeplift(test_loader, model, device, num_batches=5):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Dictionary to store cumulative DeepLIFT attributions for each layer
    cumul_dl_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs, target_class = inputs.to(device), target_class.to(device)

        # Compute the attributions for Conv2d layers
        for layer_name, layer_module in model.named_modules():
            # Skip the whole model's container and focus on Conv2d layers with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerDeepLift with the current layer
                ldl = LayerDeepLift(model, layer_module)

                # Compute the attributions for the current layer
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))

                # Sum up the attributions for the current layer
                attr_sum = attributions_ldl.cpu().data.numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_dl_attributions:
                    cumul_dl_attributions[layer_name] += attr_sum
                else:
                    cumul_dl_attributions[layer_name] = attr_sum

                del attributions_ldl, ldl

    # Calculate and print the average attributions for each layer
    avg_dl_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_dl_attributions.items()}
    for layer_name, avg_attr in avg_dl_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_dl_attributions

# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [56]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [57]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [58]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [59]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    print("deeplift")
    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("integrated_gradient")
    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    return ig_attributions, dl_attributions, train_accuracy, test_accuracy

# Functions for saving attribution

In [60]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            # Prepare data for checking if it's already processed
            combo_check = combo.copy()
            combo_check['run'] = i

            # Convert combo_check to string representation
            combo_check = {k: str(v) for k, v in combo_check.items()}



            if not existing_data.empty:
                # Filter existing_data to only the columns of interest
                filtered_data = existing_data[list(combo_check.keys())]
                # Convert filtered_data to string representation
                filtered_data = filtered_data.astype(str)
                # Check if a row with the same combination exists
                if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                    print("Combination already processed, skipping...")
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
            random.seed(i)
            np.random.seed(i)
            torch.manual_seed(i)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(i)

            train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
            ig_attributions, dl_attributions, train_accuracy, test_accuracy = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs)
            for method in ['deeplift', 'integrated_gradients']:
              if method == 'deeplift':
                  attr = dl_attributions, train_accuracy, test_accuracy
              elif method == 'integrated_gradients':
                  attr = ig_attributions, train_accuracy, test_accuracy

              combo_results = combo.copy()
              for d in attr:
                  combo_results.update(d)

              combo_results['method'] = method
              combo_results['run'] = i

              # Convert all values in combo_results to strings
              combo_results = {k: str(v) for k, v in combo_results.items()}

              # Append results to the existing data
              existing_data = existing_data.append(combo_results, ignore_index=True)

            # Save the data to CSV after each run
            existing_data.to_csv(csv_file, index=False)

# Experiments

In [61]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.0007, 0.0003],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1],
    'train_set_shuffle': [True],
    'train_batch_size': [64, 32, 16, 8]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 8}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train

In [ ]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Epoch 0: Train Loss = 1.6729, Train Accuracy = 61.33%
Epoch 0: Test Loss = 1.4987, Test Accuracy = 63.53%
Epoch 1: Train Loss = 0.5202, Train Accuracy = 85.73%
Epoch 1: Test Loss = 1.4524, Test Accuracy = 69.56%
Epoch 2: Train Loss = 0.3383, Train Accuracy = 90.54%
Epoch 2: Test Loss = 1.4525, Test Accuracy = 72.35%
deeplift
Layer: branch1.0
Average Attribution: 140.3552978515625
Layer: branch2.0
Average Attribution: 47.48756103515625
Layer: branch3.0
Average Attribution: 187.554052734375
Layer: branch4.0
Average Attribution: 235.1039794921875
integrated_gradient
Layer: branch1.0
Average Attribution: 140.2473542349239
Layer: branch2.0
Average Attribution: 47.48566579914099
Layer: branch3.0
Average Attribution: 187.57630803657605
Layer: branch4.0
Average Attribution: 235.10399200149408
runti

<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 1.6623, Train Accuracy = 60.46%
Epoch 0: Test Loss = 1.7124, Test Accuracy = 59.36%
Epoch 1: Train Loss = 0.5841, Train Accuracy = 83.48%
Epoch 1: Test Loss = 1.7051, Test Accuracy = 64.72%
Epoch 2: Train Loss = 0.3864, Train Accuracy = 89.19%
Epoch 2: Test Loss = 1.6405, Test Accuracy = 67.74%
deeplift
Layer: branch1.0
Average Attribution: 46.81512451171875
Layer: branch2.0
Average Attribution: 350.067431640625
Layer: branch3.0
Average Attribution: 72.94874267578125
Layer: branch4.0
Average Attribution: 225.7420654296875
integrated_gradient
Layer: branch1.0
Average Attribution: 46.36841892344081
Layer: branch2.0
Average Attribution: 350.0016569336277
Layer: branch3.0
Average Attribution: 72.88503881532964
Layer: branch4.0
Average Attribution: 225.7420734872456
runtime2


<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 1.6324, Train Accuracy = 60.83%
Epoch 0: Test Loss = 1.8637, Test Accuracy = 58.56%
Epoch 1: Train Loss = 0.5675, Train Accuracy = 84.26%
Epoch 1: Test Loss = 1.4833, Test Accuracy = 67.52%
Epoch 2: Train Loss = 0.3995, Train Accuracy = 88.63%
Epoch 2: Test Loss = 1.4556, Test Accuracy = 69.36%
deeplift
Layer: branch1.0
Average Attribution: 238.01650390625
Layer: branch2.0
Average Attribution: 79.80361328125
Layer: branch3.0
Average Attribution: -44.30540466308594
Layer: branch4.0
Average Attribution: 212.6083984375
integrated_gradient
Layer: branch1.0
Average Attribution: 238.14341954838352
Layer: branch2.0
Average Attribution: 79.70030632568105
Layer: branch3.0
Average Attribution: -44.244131089786315
Layer: branch4.0
Average Attribution: 212.60838939250684
runtime3


...

<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 1.4346, Train Accuracy = 62.56%
Epoch 0: Test Loss = 1.4835, Test Accuracy = 65.40%
Epoch 1: Train Loss = 0.5772, Train Accuracy = 83.27%
Epoch 1: Test Loss = 1.4863, Test Accuracy = 70.08%
Epoch 2: Train Loss = 0.4059, Train Accuracy = 88.23%
Epoch 2: Test Loss = 1.5732, Test Accuracy = 71.11%
deeplift
Layer: branch1.0
Average Attribution: 60.4442626953125
Layer: branch2.0
Average Attribution: 639.10869140625
Layer: branch3.0
Average Attribution: 122.55059814453125
Layer: branch4.0
Average Attribution: 199.1879150390625
integrated_gradient
Layer: branch1.0
Average Attribution: 60.31295163088235
Layer: branch2.0
Average Attribution: 639.0030699081276
Layer: branch3.0
Average Attribution: 122.367135204549
Layer: branch4.0
Average Attribution: 199.18792166704583
runtime14


<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 1.2831, Train Accuracy = 65.33%
Epoch 0: Test Loss = 1.3786, Test Accuracy = 63.41%
Epoch 1: Train Loss = 0.5233, Train Accuracy = 84.74%
Epoch 1: Test Loss = 1.4276, Test Accuracy = 69.65%
Epoch 2: Train Loss = 0.3572, Train Accuracy = 89.40%
Epoch 2: Test Loss = 1.6286, Test Accuracy = 67.66%
deeplift
Layer: branch1.0
Average Attribution: 144.57789306640626
Layer: branch2.0
Average Attribution: 80.9892578125
Layer: branch3.0
Average Attribution: -36.717352294921874
Layer: branch4.0
Average Attribution: 185.334033203125
integrated_gradient
Layer: branch1.0
Average Attribution: 144.8024424110601
Layer: branch2.0
Average Attribution: 81.0235955391576
Layer: branch3.0
Average Attribution: -36.93164504677198
Layer: branch4.0
Average Attribution: 185.33405096030768


<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-60-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
